<a href="https://colab.research.google.com/github/PramodModi/imagecaption/blob/master/imageCaption.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os

import pickle
import glob
import pandas as pd
import numpy as np

import seaborn as sns 
import matplotlib.pyplot as plt

import cv2
from keras.models import Model, Sequential
from tensorflow.keras.layers import Dense,Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.applications.resnet50 import ResNet50


from sklearn.model_selection import train_test_split

In [4]:
data =pickle.load(open("/content/drive/MyDrive/Colab Notebooks/ImageCaption/Data/set_1.pkl", "rb"))

print(len(data))
print(data[0])

25000
2089122314_40d5739aef.jpg#3	A fluffy , tri-colored dog be run down a road with a paper in its mouth .


In [5]:
main_df=pd.DataFrame()

In [6]:

for line in data:
    sent="START "
    word1,word2=line.split("\t")
    pic,index=word1.split('#')
    clean_sentence=word2.replace("[^a-zA-Z0-9]"," ")
    sent+=clean_sentence[:-3]
    sent+=" END"
    #print(sent)
    df=pd.DataFrame([[pic,sent]],columns=['image','description'])
    #print(df)
    if main_df.empty:
        main_df=df
    else:
        main_df=pd.concat([main_df,df])

In [7]:
main_df.head()

,image,description
0,2089122314_40d5739aef.jpg,"START A fluffy , tri-colored dog be run down a..."
0,1295671216_cde1b9c9d1.jpg,START A crowd of person be stand together on a...
0,3183195185_cd0ff994a1.jpg,"START The man and boy sit on the ground , next..."
0,2949353587_64c54e9589.jpg,START A brown dog be play with a white fluffy ...
0,361092202_3d70144ebd.jpg,START A man hike with a large backpack and two...


In [20]:
main_df.to_csv('/content/drive/MyDrive/Colab Notebooks/ImageCaption/Data/desc.csv',index=False)

In [9]:
! unzip "/content/drive/MyDrive/Colab Notebooks/ImageCaption/Data/Flicker8k_Dataset.zip"

Streaming output truncated to the last 5000 lines.
  inflating: Flicker8k_Dataset/2844846111_8c1cbfc75d.jpg  
  inflating: __MACOSX/Flicker8k_Dataset/._2844846111_8c1cbfc75d.jpg  
  inflating: Flicker8k_Dataset/1082252566_8c79beef93.jpg  
  inflating: __MACOSX/Flicker8k_Dataset/._1082252566_8c79beef93.jpg  
  inflating: Flicker8k_Dataset/1386251841_5f384a0fea.jpg  
  inflating: __MACOSX/Flicker8k_Dataset/._1386251841_5f384a0fea.jpg  
  inflating: Flicker8k_Dataset/2196316998_3b2d63f01f.jpg  
  inflating: __MACOSX/Flicker8k_Dataset/._2196316998_3b2d63f01f.jpg  
  inflating: Flicker8k_Dataset/3106791484_13e18c33d8.jpg  
  inflating: __MACOSX/Flicker8k_Dataset/._3106791484_13e18c33d8.jpg  
  inflating: Flicker8k_Dataset/3603064161_a8f3b6455d.jpg  
  inflating: __MACOSX/Flicker8k_Dataset/._3603064161_a8f3b6455d.jpg  
  inflating: Flicker8k_Dataset/2293149170_38fb2257ea.jpg  
  inflating: __MACOSX/Flicker8k_Dataset/._2293149170_38fb2257ea.jpg  
  inflating: Flicker8k_Dataset/2451114871_8617

In [5]:
!rm -rf "/content/__MACOSX"

In [11]:
SIZE = 224
image_dir = "/content/Flicker8k_Dataset/"


In [12]:

files=os.listdir(image_dir)
images=[]
name=[]
for file in files:
  name.append(file)
  img = cv2.imread(image_dir+"/"+ file, cv2.IMREAD_COLOR)
  img = cv2.resize(img, (SIZE, SIZE))
  img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
  images.append(img)


In [14]:
images_count = len(images)
print(images_count)

8091


In [15]:
name_np=np.array(name)

images_np=np.array(images)

In [16]:
np.save("/content/drive/MyDrive/Colab Notebooks/ImageCaption/Data/Images_record.npy",images_np)
np.save("/content/drive/MyDrive/Colab Notebooks/ImageCaption/Data/Name_record.npy",name_np)

In [17]:
images=np.load("/content/drive/MyDrive/Colab Notebooks/ImageCaption/Data/Images_record.npy")
names=np.load("/content/drive/MyDrive/Colab Notebooks/ImageCaption/Data/Name_record.npy")

In [18]:
len(images)

8091

In [19]:
len(names)

8091

In [21]:
df=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/ImageCaption/Data/desc.csv")
df.head()


,image,description
0,2089122314_40d5739aef.jpg,"START A fluffy , tri-colored dog be run down a..."
1,1295671216_cde1b9c9d1.jpg,START A crowd of person be stand together on a...
2,3183195185_cd0ff994a1.jpg,"START The man and boy sit on the ground , next..."
3,2949353587_64c54e9589.jpg,START A brown dog be play with a white fluffy ...
4,361092202_3d70144ebd.jpg,START A man hike with a large backpack and two...


In [22]:
## Applying Preprocessing to the language
## Removing any character which is not alphabet and whose length is not greater than 1
## Converting all words to lower case
import re
def preprocess(text, remove_digits=True):
    pattern=r'[^a-zA-z0-9\s]'
    text=re.sub(pattern,'',text)
    words=text.split()
    words_sm=[word.lower() for word in words]

    words_la=[word for word in words_sm if len(word)>1]

    words_f=[word for word in words_la if word.isalpha()]

    text=""

    for word in words_f:
      text=text+word+" "

    return text

In [23]:
df['description']=df['description'].apply(preprocess)
df.head()

,image,description
0,2089122314_40d5739aef.jpg,start fluffy tricolored dog be run down road w...
1,1295671216_cde1b9c9d1.jpg,start crowd of person be stand together on sid...
2,3183195185_cd0ff994a1.jpg,start the man and boy sit on the ground next t...
3,2949353587_64c54e9589.jpg,start brown dog be play with white fluffy stuf...
4,361092202_3d70144ebd.jpg,start man hike with large backpack and two pol...


In [25]:

## Creating a dictionary which has the image file name as key
## The dictionary has the corresponding descriptions of the files
## And the corresponding images as numpy arrays.

mapping={}
i=0
while i<len(df):
  if df.iloc[i]['image'] in mapping.keys():
    mapping[df.iloc[i]['image']]['desc'].append(df.iloc[i]['description'])
  else:
    mapping[df.iloc[i]['image']]={}
    mapping[df.iloc[i]['image']]['desc']=[df.iloc[i]['description']]
    
  i+=1

name_l=list(names)
keys=[key for key in mapping.keys()]
for key in keys:
  try:
    ind=name_l.index(key)
    mapping[key]['image']=images[ind]
  except:
    mapping.pop(key,None)


In [26]:
# Print one key and value of maping
k = list(mapping.keys())[0]
v = mapping[k]
print(k)
print(v)

2089122314_40d5739aef.jpg
{'desc': ['start fluffy tricolored dog be run down road with paper in its mout end ', 'start dog run with newspaper in its mout end ', 'start dog with newspaper in his mouth run down paved pat end '], 'image': array([[[123, 143,  78],
        [128, 150,  86],
        [126, 149,  87],
        ...,
        [143, 142, 140],
        [146, 145, 141],
        [146, 145, 141]],

       [[130, 147,  84],
        [132, 147,  87],
        [131, 145,  87],
        ...,
        [145, 145, 143],
        [146, 145, 141],
        [145, 144, 140]],

       [[135, 150,  88],
        [143, 153,  96],
        [137, 145,  91],
        ...,
        [146, 145, 141],
        [144, 143, 139],
        [144, 143, 139]],

       ...,

       [[ 74,  78,  79],
        [ 73,  81,  80],
        [ 75,  76,  78],
        ...,
        [ 97,  97,  96],
        [ 91,  91,  93],
        [ 87,  87,  89]],

       [[ 69,  74,  75],
        [ 75,  83,  82],
        [ 78,  80,  82],
        ...,
   

In [30]:
# Save imgae array and decriptin 
X =[]
y=[]
for k in mapping.keys():
  X.append(mapping[k]['image'])
  y.append(mapping[k]['desc'])

X_f=np.array(X)
y_f=np.array(y)



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  if __name__ == '__main__':


In [31]:
print(len(X_f))
print(len(y_f))

8032
8032


In [8]:
train_images = np.array(train_images)
trina_images_p1 = train_images[0:6000]
print(len(trina_images_p1))




6000


In [9]:
trina_images_p1 = trina_images_p1/255.0

In [ ]:
np.delete(train_images, len(train_images))

In [ ]:

train_images_p2 = train_images[6000:images_count]
print(len(train_images_p2))
train_images_p2 = train_images_p2/255.0

In [22]:
Resnet50_model = ResNet50(weights= 'imagenet', include_top = False, input_shape = (SIZE, SIZE,3))

94781440/94765736 [==============================] - 1s 0us/step


In [23]:
Resnet50_model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 262, 262, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 128, 128, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 128, 128, 64) 256         conv1_conv[0][0]                 
___________________________________________________________________________________________

In [26]:
# As this model is already trained, and we are going to use train model for feature generator. So, we will not train the model.
for layer in Resnet50_model.layers:
  layer.trainable = False

In [27]:
Resnet50_model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 262, 262, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 128, 128, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 128, 128, 64) 256         conv1_conv[0][0]                 
___________________________________________________________________________________________

**As shown above, o trainable parameters.**


In [28]:
# Now, extract the features
feature_extractor = Resnet50_model.predict(train_images_arr)

AttributeError: ignored